In [2]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [7]:
from langchain_community.utilities import SQLDatabase

In [8]:
db = SQLDatabase.from_uri("sqlite:///training.db")

print(db.dialect)
print(db.get_usable_table_names())

db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [9]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

In [10]:
question = "How many employees are there?"

response = chain.invoke({
    "question": question
})

response

'SELECT COUNT("EmployeeId") AS TotalEmployees\nFROM "Employee"'

In [11]:
db.run(response)

'[(8,)]'

In [12]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [17]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.tools import tool

# In Some of the LLM Models, they generate SQL Statement with JSON Output
# as SQLQuery: 'SELECT * FROM EMPLOYEES'

@tool
def parse(queryString: str) -> str:
    """
        Parses Generated SQL Statement
    """

    splitted = queryString.split(':')

    if (len(splitted) >= 2):
        query = splitted[1]
    else:
        query = queryString

    return query

In [18]:
parse('SELECT COUNT("EmployeeId") AS TotalEmployees\nFROM "Employee"')

'SELECT COUNT("EmployeeId") AS TotalEmployees\nFROM "Employee"'

In [19]:
execute_query = QuerySQLDataBaseTool(db = db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | parse | execute_query

question = "How many employees are there?"

response = chain.invoke({
    "question": question
})

response

'[(8,)]'

In [20]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding SQL Query, and SQL Result, answer the user question
    
    Question: {question}
    SQL Query: {query}
    Result: {result}
    Answer:
    """
)

chain = RunnablePassthrough.assign(query = write_query) \
    .assign(result = itemgetter("query") | parse | execute_query) | \
    answer_prompt | llm | StrOutputParser()
    
question = "How many employees are there?"

response = chain.invoke({
    "question": question
})

response

'There are 8 employees.'

In [21]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7879abf2c050>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7879abf2c050>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7879abf2c050>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [22]:
%pip install --upgrade --quiet langgraph


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [24]:
from langchain_core.messages import SystemMessage

SQL_PREFIX ="""You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content = SQL_PREFIX)

In [26]:
agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

In [29]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content = "Which country's customers have spent the most?")
    ]
}):
    print(stream)
    print("******")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4jvwagtoH6I9mxwnsvbS0dA2', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 558, 'total_tokens': 570}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f22286e3-8951-4d4c-989e-cb745150bd70-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_4jvwagtoH6I9mxwnsvbS0dA2'}], usage_metadata={'input_tokens': 558, 'output_tokens': 12, 'total_tokens': 570})]}}
******
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='call_4jvwagtoH6I9mxwnsvbS0dA2')]}}
******
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0GFiYkCziI1V

In [30]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content = "Describe the playlisttrack table")
    ]
}):
    print(stream)
    print("******")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jilhtPU9y3c9TqlSjHGZcrye', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 554, 'total_tokens': 571}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b0228278-9916-4261-9ec5-8a4a091a839a-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_jilhtPU9y3c9TqlSjHGZcrye'}], usage_metadata={'input_tokens': 554, 'output_tokens': 17, 'total_tokens': 571})]}}
******
{'tools': {'messages': [ToolMessage(content="Error: table_names {'playlisttrack'} not found in database", name='sql_db_schema', tool_call_id='call_jilhtPU9y3c9TqlSjHGZcrye')]}}
******
{'agent': {'messages': [AIMessage(content='I apologize for the error. Let me first retrieve the list of tabl

In [31]:
import ast
import re


def query_as_list(database, query):
    result = db.run(query)
    result = [el for sub in ast.literal_eval(result) for el in sub if el]
    result = [re.sub(r"\b\d+\b", "", string).strip() for string in result]

    return list(set(result))

artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")



In [33]:
artists[:5]

['Wilhelm Kempff', 'Peter Tosh', 'A Cor Do Som', 'Jack Johnson', 'JET']

In [34]:
albums[:5]

['Chronicle, Vol.',
 'Vs.',
 'Allegri: Miserere',
 'Os Cães Ladram Mas A Caravana Não Pára',
 'Realize']

In [35]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [40]:
vector_database = FAISS.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_database.as_retriever(search_kwargs = {"k": 5})
description = """
    use to lookup values to filter on.
    
    Input is an approximate spelling of the valid proper nouns.
    
    Use the noun most similar to the search
"""

retriever_tool = create_retriever_tool(retriever, name = "search_proper_nouns", description=description)

In [42]:
print(retriever_tool.invoke("Alis Chains"))

Alice In Chains

Aisha Duo

Alanis Morissette

Slaves And Masters

Xis
